In [1]:
cd ../

C:\Users\thyne\Documents\GitHub\thyne-lux


You can upload the car dataset to a local postgres database using the upload_car_data.py script in the lux/data folder. You will need to update the name of the database and the login credentials in that file.

In [2]:
import lux
import psycopg2
import pandas as pd

connection = psycopg2.connect("host=localhost dbname=postgres_db user=postgres password=lux")

sql_df = lux.LuxDataFrame()
sql_df.set_SQL_connection(connection, "car")
sql_df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [5]:
sql_df.data_type

{'quantitative': ['milespergal',
  'displacement',
  'horsepower',
  'weight',
  'acceleration'],
 'ordinal': [],
 'nominal': ['name', 'cylinders', 'origin'],
 'temporal': ['year'],
 'id': []}

In [3]:
sql_df.set_intent(["milespergal"])
sql_df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [6]:
from lux.vis.Vis import Vis
from lux.vis.Vis import Clause

x_clause = Clause(attribute = "milespergal", channel = "x")
y_clause = Clause(attribute = "weight", channel = "y")
color_clause = Clause(attribute = 'cylinders', channel = "color")
filter_clause = Clause(attribute ="horsepower", filter_op=">", value=150)

new_vis = Vis([x_clause, y_clause, color_clause])
new_vis.refresh_source(sql_df)
new_vis

LuxWidget(current_vis={'config': {'view': {'continuousWidth': 400, 'continuousHeight': 300}, 'axis': {'labelCo…

<Vis  (x: milespergal, y: weight, color: cylinders) mark: scatter, score: 0.0 >